# LSTM Classification with SUBJ Dataset
<hr>

We will build a text classification model using LSTM model on the SUBJ Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


,sentence,label,split
0,"smart and alert , thirteen conversations about...",0,train
1,"color , musical bounce and warm seas lapping o...",0,train
2,it is not a mass market entertainment but an u...,0,train
3,a light hearted french film about the spiritua...,0,train
4,my wife is an actress has its moments in looki...,0,train
...,...,...,...
9995,"in the end , they discover that balance in lif...",1,train
9996,a counterfeit 1000 tomin bank note is passed i...,1,train
9997,enter the beautiful and mysterious secret agen...,1,train
9998,after listening to a missionary from china spe...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5000,5000
1,5000,5000


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## LSTM Model

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 585,825
Trainable params: 585,825
Non-trainable params: 0
_________________________________________________________________


In [13]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [14]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
282/282 [==============================] - 130s 397ms/step - loss: 0.4078 - accuracy: 0.8065 - val_loss: 0.1930 - val_accuracy: 0.9210
Epoch 2/15
282/282 [==============================] - 96s 340ms/step - loss: 0.0618 - accuracy: 0.9814 - val_loss: 0.2342 - val_accuracy: 0.9110
Epoch 3/15
282/282 [==============================] - 111s 394ms/step - loss: 0.0137 - accuracy: 0.9967 - val_loss: 0.4000 - val_accuracy: 0.9060
Epoch 4/15
282/282 [==============================] - 113s 401ms/step - loss: 0.0116 - accuracy: 0.9965 - val_loss: 0.5076 - val_accuracy: 0.8960
Epoch 5/15
282/282 [==============================] - 109s 385ms/step - loss: 0.0127 - accuracy: 0.9953 - val_loss: 0.5245 - val_accuracy: 0.8930
Epoch 6/15
282/282 [==============================] - 114s 405ms/step - loss: 0.0049 - accuracy: 0.9983 - val_loss: 0.6017 - val_accuracy: 0.9060
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 92.1000003814

Epoch 5/15
282/282 [==============================] - 266s 945ms/step - loss: 0.0034 - accuracy: 0.9988 - val_loss: 0.5366 - val_accuracy: 0.8840
Epoch 6/15
282/282 [==============================] - 265s 940ms/step - loss: 0.0052 - accuracy: 0.9979 - val_loss: 0.5926 - val_accuracy: 0.8860
Epoch 7/15
282/282 [==============================] - 265s 940ms/step - loss: 0.0052 - accuracy: 0.9987 - val_loss: 0.6639 - val_accuracy: 0.8920
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 90.20000100135803
Training 9: 
Epoch 1/15
282/282 [==============================] - 190s 596ms/step - loss: 0.4161 - accuracy: 0.8121 - val_loss: 0.1984 - val_accuracy: 0.9200
Epoch 2/15
282/282 [==============================] - 169s 600ms/step - loss: 0.0561 - accuracy: 0.9829 - val_loss: 0.2353 - val_accuracy: 0.9080
Epoch 3/15
282/282 [==============================] - 204s 723ms/step - loss: 0.0114 - accuracy: 0.9964 - val_loss: 0.3332 - val_accuracy: 0

## Summary

In [15]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,92.1,91.2,91.799998,91.399997,91.900003,93.099999,92.500001,90.200001,92.000002,92.6,91.88


In [16]:
report = record
report = report.to_excel('LSTM_SUBJ_v2.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [17]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [18]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [19]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [20]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [21]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [22]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.77412221, -0.66578379, -2.36541218, ..., -0.07308953,
        -1.27580538,  1.21141937],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## LSTM Model

In [23]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [24]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 585,825
Trainable params: 285,825
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [25]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [26]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/40
282/282 [==============================] - 123s 359ms/step - loss: 0.4387 - accuracy: 0.7690 - val_loss: 0.4515 - val_accuracy: 0.8330
Epoch 2/40
282/282 [==============================] - 87s 310ms/step - loss: 0.2242 - accuracy: 0.9135 - val_loss: 0.5378 - val_accuracy: 0.8240
Epoch 3/40
282/282 [==============================] - 94s 334ms/step - loss: 0.1630 - accuracy: 0.9394 - val_loss: 0.7060 - val_accuracy: 0.8000
Epoch 4/40
282/282 [==============================] - 97s 343ms/step - loss: 0.1343 - accuracy: 0.9542 - val_loss: 0.9440 - val_accuracy: 0.7750
Epoch 5/40
282/282 [==============================] - 96s 341ms/step - loss: 0.0976 - accuracy: 0.9642 - val_loss: 0.9601 - val_accuracy: 0.7900
Epoch 6/40
282/282 [==============================] - 96s 341ms/step - loss: 0.0754 - accuracy: 0.9749 - val_loss: 1.0075 - val_accuracy: 0.8070
Epoch 7/40
282/282 [==============================] - 95s 337ms/step - loss: 0.0546 - accuracy: 0.9816 - val_loss: 1

282/282 [==============================] - 258s 915ms/step - loss: 0.0113 - accuracy: 0.9965 - val_loss: 0.4335 - val_accuracy: 0.9210
Epoch 18/40
282/282 [==============================] - 260s 920ms/step - loss: 0.0026 - accuracy: 0.9993 - val_loss: 0.3926 - val_accuracy: 0.9180
Epoch 19/40
282/282 [==============================] - 259s 919ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.4223 - val_accuracy: 0.9150
Epoch 20/40
282/282 [==============================] - 258s 915ms/step - loss: 0.0163 - accuracy: 0.9947 - val_loss: 0.4481 - val_accuracy: 0.9070
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Test Accuracy: 93.09999942779541
Training 5: 
Epoch 1/40
282/282 [==============================] - 235s 741ms/step - loss: 0.4323 - accuracy: 0.7867 - val_loss: 0.3185 - val_accuracy: 0.8560
Epoch 2/40
282/282 [==============================] - 166s 589ms/step - loss: 0.2259 - accuracy: 0.9089 - val_loss: 0.2484 - val_accuracy: 0.8930
Ep

Test Accuracy: 85.19999980926514
Training 8: 
Epoch 1/40
282/282 [==============================] - 271s 894ms/step - loss: 0.4333 - accuracy: 0.7831 - val_loss: 0.2458 - val_accuracy: 0.8960
Epoch 2/40
282/282 [==============================] - 299s 1s/step - loss: 0.2195 - accuracy: 0.9109 - val_loss: 0.3002 - val_accuracy: 0.8680
Epoch 3/40
282/282 [==============================] - 330s 1s/step - loss: 0.1835 - accuracy: 0.9260 - val_loss: 0.2431 - val_accuracy: 0.8920
Epoch 4/40
282/282 [==============================] - 335s 1s/step - loss: 0.1421 - accuracy: 0.9484 - val_loss: 0.3320 - val_accuracy: 0.8770
Epoch 5/40
282/282 [==============================] - 327s 1s/step - loss: 0.1166 - accuracy: 0.9599 - val_loss: 0.4545 - val_accuracy: 0.8600
Epoch 6/40
282/282 [==============================] - 323s 1s/step - loss: 0.0875 - accuracy: 0.9705 - val_loss: 0.4032 - val_accuracy: 0.8700
Epoch 7/40
282/282 [==============================] - 323s 1s/step - loss: 0.0753 - accuracy:

## Summary

In [27]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,83.3,89.700001,88.999999,93.099999,91.7,91.799998,85.2,89.600003,91.2,87.5,89.21


In [28]:
report = record2
report = report.to_excel('LSTM_SUBJ_v2_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## LSTM Model

In [29]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [30]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_44 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_45 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 129       
Total params: 585,825
Trainable params: 585,825
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [33]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [34]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 
Epoch 1/40
282/282 [==============================] - 461s 2s/step - loss: 0.3894 - accuracy: 0.8028 - val_loss: 0.2029 - val_accuracy: 0.9150
Epoch 2/40
282/282 [==============================] - 361s 1s/step - loss: 0.0968 - accuracy: 0.9659 - val_loss: 0.2166 - val_accuracy: 0.9090
Epoch 3/40
282/282 [==============================] - 421s 1s/step - loss: 0.0282 - accuracy: 0.9902 - val_loss: 0.3713 - val_accuracy: 0.8990
Epoch 4/40
282/282 [==============================] - 441s 2s/step - loss: 0.0158 - accuracy: 0.9960 - val_loss: 0.3699 - val_accuracy: 0.9080
Epoch 5/40
282/282 [==============================] - 440s 2s/step - loss: 0.0043 - accuracy: 0.9990 - val_loss: 0.4967 - val_accuracy: 0.8990
Epoch 6/40
282/282 [==============================] - 437s 2s/step - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.5952 - val_accuracy: 0.8970
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 91.50000214576721
Training 2: 

Epoch 5/40
282/282 [==============================] - 395s 1s/step - loss: 0.0065 - accuracy: 0.9977 - val_loss: 0.5661 - val_accuracy: 0.8850
Epoch 6/40
282/282 [==============================] - 395s 1s/step - loss: 0.0012 - accuracy: 0.9995 - val_loss: 0.7314 - val_accuracy: 0.8870
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 91.00000262260437
Training 9: 
Epoch 1/40
282/282 [==============================] - 500s 2s/step - loss: 0.3828 - accuracy: 0.8127 - val_loss: 0.2447 - val_accuracy: 0.9020
Epoch 2/40
282/282 [==============================] - 442s 2s/step - loss: 0.0854 - accuracy: 0.9709 - val_loss: 0.2902 - val_accuracy: 0.8870
Epoch 3/40
282/282 [==============================] - 436s 2s/step - loss: 0.0271 - accuracy: 0.9926 - val_loss: 0.4460 - val_accuracy: 0.8890
Epoch 4/40
282/282 [==============================] - 435s 2s/step - loss: 0.0177 - accuracy: 0.9949 - val_loss: 0.4484 - val_accuracy: 0.8840
Epoch 5/40
2

## Summary

In [35]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,91.500002,92.199999,89.999998,92.1,92.699999,92.400002,88.700002,91.000003,90.200001,91.600001,91.240001


In [36]:
report = record3
report = report.to_excel('LSTM_SUBJ_v2_3.xlsx', sheet_name='dynamic')